<h1>Python для анализа данных.</h1><h2>Домашняя работа №1</h2>

In [1]:
#Подключим необходимые библиотеки
import numpy as np
import pandas as pd

<h3>Содержимое файлов:</h3>

<b>lectures.csv:</b></br>
Метаданные лекций, просмотренных пользователями по мере продвижения своего образования.</br>
+ lecture_id - внешний ключ для столбца train/test content_id, когда типом контента является lecture (1);
+ part - код категории верхнего уровня для лекции;
+ tag - один из тегов содержит коды для лекции. Значение тегов не будет указано, но этих кодов достаточно для объединения лекций в группы;
+ type_of - краткое описание основной цели лекции.

<hr>
<b>questions.csv:</b></br>
Метаданные для ответов на вопросы, задаваемые пользователям.</br>

+ question_id - внешний ключ для столбца train/test content_id, когда типом контента является question (0);
+ bundle_id - код, по которому вопросы подаются вместе;
+ correct_answer - Ответ на вопрос. Можно сравнить со столбцом train user_answer, чтобы проверить, был ли пользователь прав;
+ part - соответствующий раздел теста TOEIC;
+ tags - один или несколько подробных кодов тегов для вопроса. Значение тегов не будет указано, но этих кодов достаточно для объединения вопросов в группы

<hr>
<b>train.csv:</b></br>
Метаданные для ответов на вопросы, задаваемые пользователям.</br>

+ row_id - идентификационный код для строки;
+ timestamp - время в миллисекундах между этим взаимодействием пользователя и завершением первого события от этого пользователя;
+ user_id - идентификационный код пользователя
+ content_type_id - 0, если событием был вопрос, заданный пользователю, 1, если событием был просмотр пользователем лекции;
+ task_container_id - идентификационный код для набора вопросов или лекций;
+ user_answer - ответ пользователя на вопрос, если таковой имеется. Считайте -1 как null, для лекций;
+ answered_correctly - если пользователь ответил правильно. Считайте -1 как null, для лекций;
+ prior_question_elapsed_time - среднее время, затраченное пользователем на решение каждого вопроса из предыдущего набора;
+ prior_question_had_explanation - Независимо от того, видел ли пользователь объяснение и правильный ответ (ы) после ответа на предыдущий набор вопросов, игнорируя любые лекции между ними.


In [2]:
#Загрузим данные questions
question_dtypes = {
    'question_id': 'int16',
    'bundle_id': 'uint16',
    'correct_answer': 'int8',
    'part': 'uint8',
}

questions = pd.read_csv('questions.csv', dtype=question_dtypes)

#Выведем первые и последние 5 строк
display(questions.head())
display(questions.tail())

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38


,question_id,bundle_id,correct_answer,part,tags
13518,13518,13518,3,5,14
13519,13519,13519,3,5,8
13520,13520,13520,2,5,73
13521,13521,13521,0,5,125
13522,13522,13522,3,5,55


In [105]:
#Загрузим данные lectures
lectures_dtypes = {
    'lecture_id': 'int64',
    'tag': 'uint16',
    'part': 'uint8',
}

lectures = pd.read_csv('lectures.csv', dtype=lectures_dtypes)

#Выведем первые и последние 5 строк
display(lectures.head())
display(lectures.tail())

,lecture_id,tag,part,type_of
0,89,159,5,concept
1,100,70,1,concept
2,185,45,6,concept
3,192,79,5,solving question
4,317,156,5,solving question


,lecture_id,tag,part,type_of
413,32535,8,5,solving question
414,32570,113,3,solving question
415,32604,24,6,concept
416,32625,142,2,concept
417,32736,82,3,concept


In [4]:
#Загрузим данные train
#Опишем словарь с типами данных для DataFrame
train_dtypes = {
    "row_id": "int64",
    "timestamp": "int64",
    "user_id": "int32",
    "content_id": "int16",
    "content_type_id": "boolean",
    "task_container_id": "int16",
    "user_answer": "int8",
    "answered_correctly": "int8",
    "prior_question_elapsed_time": "float32", 
    "prior_question_had_explanation": "boolean"
}
#Создадим список в котором будут части DataFrame train
train_parts_list = list()

#Так как файл слишком большой, будем читать частями и полученные части записывать в список
for chunck in pd.read_csv('train.csv', dtype=train_dtypes, chunksize = 500000):
    #Запишем в список кусок DataFrame train
    train_parts_list.append(chunck)

#Из частей DataFrame соберем единый DataFrame train
train = pd.concat(train_parts_list, ignore_index=True)

#Освободим память списка
train_parts_list = list()

#Выведем первые и последние 5 строк
display(train.head())
display(train.tail())

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,False,1,3,1,NaN,<NA>
1,1,56943,115,5716,False,2,2,1,37000.0,False
2,2,118363,115,128,False,0,0,1,55000.0,False
3,3,131167,115,7860,False,3,0,1,19000.0,False
4,4,137965,115,7922,False,4,1,1,11000.0,False


,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
101230327,101230327,428564420,2147482888,3586,False,22,0,1,18000.0,True
101230328,101230328,428585000,2147482888,6341,False,23,3,1,14000.0,True
101230329,101230329,428613475,2147482888,4212,False,24,3,1,14000.0,True
101230330,101230330,428649406,2147482888,6343,False,25,1,0,22000.0,True
101230331,101230331,428692118,2147482888,7995,False,26,3,1,29000.0,True


Получим информацию о таблице questions

In [5]:
questions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13523 entries, 0 to 13522
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   question_id     13523 non-null  int16 
 1   bundle_id       13523 non-null  uint16
 2   correct_answer  13523 non-null  int8  
 3   part            13523 non-null  uint8 
 4   tags            13522 non-null  object
dtypes: int16(1), int8(1), object(1), uint16(1), uint8(1)
memory usage: 185.0+ KB


Из данной информации видно:</br>

+ Общее количество строк 13 523
+ Общее количество полей 5
+ Объем занимаемой памяти 528.4 KB
+ В поле tags пропущено 1 значение

In [6]:
print(questions[pd.isnull(questions).any(axis=1)])

       question_id  bundle_id  correct_answer  part tags
10033        10033      10033               2     6  NaN


<hr>
Получим информацию о таблице lectures

In [7]:
lectures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   lecture_id  418 non-null    uint32
 1   tag         418 non-null    uint16
 2   part        418 non-null    uint8 
 3   type_of     418 non-null    object
dtypes: object(1), uint16(1), uint32(1), uint8(1)
memory usage: 6.3+ KB


Из данной информации видно:</br>

+ Общее количество строк 418
+ Общее количество полей 4
+ Объем занимаемой памяти 13.2 KB
+ Нет пропущенных значений

<HR>
Получим информацию о таблице train

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101230332 entries, 0 to 101230331
Data columns (total 10 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   row_id                          int64  
 1   timestamp                       int64  
 2   user_id                         int32  
 3   content_id                      int16  
 4   content_type_id                 boolean
 5   task_container_id               int16  
 6   user_answer                     int8   
 7   answered_correctly              int8   
 8   prior_question_elapsed_time     float32
 9   prior_question_had_explanation  boolean
dtypes: boolean(2), float32(1), int16(2), int32(1), int64(2), int8(2)
memory usage: 3.2 GB


Из данной информации видно:</br>

+ Общее количество строк 101 230 332
+ Общее количество полей 10
+ Объем занимаемой памяти 3.2 GB
+ Сведения о пропущенных значениях не отображено, так как DataFrame слишком велик</br>

Посчитаем количество пропиущенных значений самостоятельно:

In [9]:
print('Количество пропущеных значений в полях:')
print('row_id -', sum(train['row_id'].isnull()))
print('timestamp -', sum(train['timestamp'].isnull()))
print('user_id -', sum(train['user_id'].isnull()))
print('content_id -', sum(train['content_id'].isnull()))
print('content_type_id -', sum(train['content_type_id'].isnull()))
print('task_container_id -', sum(train['task_container_id'].isnull()))
print('user_answer -', sum(train['user_answer'].isnull()))
print('answered_correctly -', sum(train['answered_correctly'].isnull()))
print('prior_question_elapsed_time -', sum(train['prior_question_elapsed_time'].isnull()))
print('prior_question_had_explanation -', sum(train['prior_question_had_explanation'].isnull()))

Количество пропущеных значений в полях:
row_id - 0
timestamp - 0
user_id - 0
content_id - 0
content_type_id - 0
task_container_id - 0
user_answer - 0
answered_correctly - 0
prior_question_elapsed_time - 2351538
prior_question_had_explanation - 392506


Пропущены значения только в полях prior_question_elapsed_time и prior_question_had_explanation
<hr>
Проанализируем общий процент правильных ответов:

In [10]:
#Получим доли ответов по всему DataFrame
correct_answers = train['answered_correctly'].value_counts(normalize=True) * 100
#Получим среднее время обучения
ts_mean = train['timestamp'].mean()
#Получим среднее время затраченное на ответ
elapsed_time_mean = train['prior_question_elapsed_time'].mean()

In [11]:
print('≈ ', round(correct_answers[0], 2), '% неправильных ответа', sep='')
print('≈ ', round(correct_answers[1], 2), '% правильных ответа', sep='')
print('≈ ', round(correct_answers[-1], 2), '% лекций', sep='')
print('Среднее время обучения:', round(ts_mean / 1000 / 60 / 60 / 24, 2), 'дней')
print('Среднее время затраченное на ответ:', round(elapsed_time_mean / 1000, 2), 'секунд')

≈ 33.61% неправильных ответа
≈ 64.45% правильных ответа
≈ 1.94% лекций
Среднее время обучения: 89.16 дней
Среднее время затраченное на ответ: 25.42 секунд


На основе данного показателя можно сделать вывод, что в большинстве случаев обучающие отвечают правильно.
<hr>
Проанализируем количество правильных ответов в зависимости от объяснения лекции

In [12]:
#Получим доли ответов по DataFrame в зависимости от объяснения лекции
#Из answered_correctly исключим -1, так как это лекция и не имеет отношения к ответам и сгруппируем по prior_question_had_explanation
explanation = train[train['answered_correctly'] != -1].groupby('prior_question_had_explanation')
#Получим долю правильных ответов
explanation_correct = explanation['answered_correctly'].value_counts(normalize=True) * 100
#Получим среднее время затраченное на ответ
explanation_elapsed_time = explanation['prior_question_elapsed_time'].mean()
#Для освобождения памяти
explanation = 0

In [13]:
print('Без Объяснения лекции:')
print('≈ ', round(explanation_correct[False, 0], 2), '% неправильных ответов', sep='')
print('≈ ', round(explanation_correct[False, 1], 2), '% правильных ответов', sep='')
print('Среднее время затраченное на ответ:', round(explanation_elapsed_time[False]/1000, 2), 'секунд')
print('\nПосле объяснения лекции:')
print('≈ ', round(explanation_correct[True, 0], 2), '% неправильных ответов', sep='')
print('≈ ', round(explanation_correct[True, 1], 2), '% правильных ответов', sep='')
print('Среднее время затраченное на ответ:', round(explanation_elapsed_time[True]/1000, 2), 'секунд')

Без Объяснения лекции:
≈ 49.94% неправильных ответов
≈ 50.06% правильных ответов
Среднее время затраченное на ответ: 25.59 секунд

После объяснения лекции:
≈ 32.68% неправильных ответов
≈ 67.32% правильных ответов
Среднее время затраченное на ответ: 25.41 секунд


На основе проделланого анализаможно сделать вывод, что успеваемость улучшается после объяснения лекции, но среднее затраченное время на ответ не меняется.
<hr>
Проанализируем как время обучения влияет на правильные ответы

In [14]:
#Разделим периоды обучения на 10 квантилей
last_qt = 0
for quant in range(1, 11):
    #Для каждого квантиля определим долю правильных ответов
    ts_qt = train['timestamp'].quantile(quant/10)
    answer_qt = train[
        (train['timestamp'] > last_qt) &
        (train['timestamp'] <= int(ts_qt)) &
        #Из answered_correctly исключим -1, так как это лекция и не имеет отношения к ответам
        (train['answered_correctly'] != -1)
    ]['answered_correctly'].value_counts(normalize=True) * 100
    print('Кванитиль ', quant, '/10. Доля правильных ответов = ', round(answer_qt[1], 2), sep='' )

Кванитиль 1/10. Доля правильных ответов = 51.73
Кванитиль 2/10. Доля правильных ответов = 58.64
Кванитиль 3/10. Доля правильных ответов = 61.21
Кванитиль 4/10. Доля правильных ответов = 62.62
Кванитиль 5/10. Доля правильных ответов = 63.6
Кванитиль 6/10. Доля правильных ответов = 64.3
Кванитиль 7/10. Доля правильных ответов = 64.8
Кванитиль 8/10. Доля правильных ответов = 65.16
Кванитиль 9/10. Доля правильных ответов = 65.47
Кванитиль 10/10. Доля правильных ответов = 65.71


На основе проделланого анализа можно сделать вывод, что с увеличением продолжительности обучения, ростет доля правильных ответов
<hr>
Выполним анализ вопросов

In [109]:
#Объеденим train DataFrame и question DataFrame
train_question = pd.merge(train[train['content_type_id'] == 0], questions, left_on='content_id', right_on='question_id')

In [110]:
#Получим доли правильных ответов в группавх
tag_answers =train_question.groupby('tags')['answered_correctly'].value_counts(normalize=True) * 100

In [111]:
#Разложим ответ на словари с правильными и неправильными ответами
positive_dict = tag_answers[:,1].to_dict()
negative_dict = tag_answers[:,0].to_dict()

print('Группа вопросов с наибольшим количеством правильных ответов:', max(positive_dict, key=positive_dict.get))
print('Группа вопросов с наибольшим количеством ошибочных ответов:', max(negative_dict, key=negative_dict.get))

#Для экономии памяти удаляем объекты
del tag_answers
del train_question

Группа вопросов с наибольшим количеством правильных ответов: 98 84 76 135
Группа вопросов с наибольшим количеством ошибочных ответов: 118 42 21 162


На основе проделанного анализа можно сделать вывод, что самая простая (усваеваемая) тема для вопросов находится под тэгом 98 84 76 135, а самая сложная (не усваеваемая) под тэгом 118 42 21 162
<hr>
Выполним анализ лекций

In [112]:
#Объеденим train DataFrame и lectures DataFrame
train_lectures = pd.merge(train[train['content_type_id'] == 1], lectures, left_on='content_id', right_on='lecture_id')

In [125]:
#Получим частотынй анализ просмотра лекций
lectures_value = train_lectures['tag'].value_counts(normalize=True) * 100
lectures_dict = lectures_value.to_dict()

print('Самая посещаемая лекция:', max(lectures_dict, key=lectures_dict.get))
print('Самая менее посещаемая лекция:', min(lectures_dict, key=lectures_dict.get))

Самая посещаемая лекция: 62
Самая менее посещаемая лекция: 137


In [132]:
lectures[lectures['tag'] == 62]

,lecture_id,tag,part,type_of
38,3153,62,2,intention
93,8288,62,2,solving question
103,8976,62,2,intention
117,10040,62,2,solving question
178,14654,62,2,solving question
203,16773,62,2,solving question
